Om de volgende notebooks uit te voeren, moet je, als je dat nog niet hebt gedaan, de openai-sleutel instellen in het .env-bestand als `OPENAI_API_KEY`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

Vervolgens gaan we de Embedding Index laden in een Pandas Dataframe. De Embedding Index is opgeslagen in een JSON-bestand genaamd `embedding_index_3m.json`. De Embedding Index bevat de Embeddings voor elk van de YouTube-transcripten tot eind oktober 2023.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Vervolgens gaan we een functie maken genaamd `get_videos` die de Embedding Index doorzoekt op basis van de zoekopdracht. De functie geeft de top 5 video's terug die het meest lijken op de zoekopdracht. De functie werkt als volgt:

1. Eerst wordt er een kopie van de Embedding Index gemaakt.
2. Daarna wordt de Embedding voor de zoekopdracht berekend met behulp van de OpenAI Embedding API.
3. Vervolgens wordt er een nieuwe kolom aangemaakt in de Embedding Index met de naam `similarity`. De kolom `similarity` bevat de cosinus-overeenkomst tussen de Embedding van de zoekopdracht en de Embedding van elk videofragment.
4. Daarna wordt de Embedding Index gefilterd op basis van de kolom `similarity`. De Embedding Index wordt zo gefilterd dat alleen video's overblijven met een cosinus-overeenkomst van 0,75 of hoger.
5. Tot slot wordt de Embedding Index gesorteerd op de kolom `similarity` en worden de top 5 video's teruggegeven.


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Deze functie is heel eenvoudig, ze print gewoon de resultaten van de zoekopdracht.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Eerst wordt de Embedding Index geladen in een Pandas Dataframe.
2. Vervolgens wordt de gebruiker gevraagd om een zoekopdracht in te voeren.
3. Daarna wordt de functie `get_videos` aangeroepen om in de Embedding Index te zoeken naar de zoekopdracht.
4. Tot slot wordt de functie `display_results` aangeroepen om de resultaten aan de gebruiker te tonen.
5. Daarna wordt de gebruiker opnieuw gevraagd om een zoekopdracht in te voeren. Dit proces herhaalt zich totdat de gebruiker `exit` invoert.

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.nl.png)

Je wordt gevraagd om een zoekopdracht in te voeren. Voer een zoekopdracht in en druk op enter. De applicatie geeft een lijst met video's terug die relevant zijn voor de zoekopdracht. De applicatie geeft ook een link naar het moment in de video waar het antwoord op de vraag te vinden is.

Hier zijn enkele zoekopdrachten om uit te proberen:

- Wat is Azure Machine Learning?
- Hoe werken convolutionele neurale netwerken?
- Wat is een neuraal netwerk?
- Kan ik Jupyter Notebooks gebruiken met Azure Machine Learning?
- Wat is ONNX?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)


---

**Disclaimer**:
Dit document is vertaald met behulp van de AI-vertalingsdienst [Co-op Translator](https://github.com/Azure/co-op-translator). Hoewel we streven naar nauwkeurigheid, dient u er rekening mee te houden dat geautomatiseerde vertalingen fouten of onnauwkeurigheden kunnen bevatten. Het originele document in de oorspronkelijke taal moet als de gezaghebbende bron worden beschouwd. Voor kritische informatie wordt professionele menselijke vertaling aanbevolen. Wij zijn niet aansprakelijk voor misverstanden of verkeerde interpretaties die voortvloeien uit het gebruik van deze vertaling.
